<a href="https://colab.research.google.com/github/DmitryKutsev/NIS_SentiFrame/blob/master/parsing_bl_pandas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%%capture
!pip install pymorphy2[fast]
import pandas as pd
import json
from collections import Counter
# from pandas.io.json import json_normalize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

In [0]:
# ! wget https://raw.githubusercontent.com/nicolay-r/RuSentiFrames/master/collection.json
with open ('collection.json') as jf:
  bl_data = json.load(jf)

In [3]:
# задаём названия колонок, хотя этого можно и не делать, наверное
cols = ['title', 'variants', 'comment', 'roles.a0', 'roles.a1', 'roles.a2', 'roles.a3',
       'frames.polarity', 'frames.effect', 'frames.state', 'variant', 'key']

# создаём шаблон итогового датафрейма с нужными колонками (см. выше)
# к нему будем приклеивать датафреймы, в которых строки будут различаться лишь
# колонкой вариант
bl_df = pd.DataFrame(columns=cols)

# для каждого фрейма
for fr in bl_data:
  # достаём фрейм
  frame = bl_data[fr]
  # создаём второй шаблон, к которому будем приклеивать строки для каждого варианта
  # когда наполним этот второй шаблон, он отправится в итоговый дф
  df = pd.DataFrame(columns=cols)


  # для каждого варианта предиката
  for variant in frame['variants']:
    # переносим джейсон фрейма, с которым работаем, в датафрейм из одной строки
    # max_level=1 потому что дальше фрейм не разворачивается
    base_df = pd.json_normalize(frame, max_level=1)
    # в новую колонку кладём вариант
    base_df['variant'] = [variant]
    # ко второму дф приклеиваем дф-строку с вариантом, индекс сбрасываем
    # не сортируем, чтоб нимношк быстрее было
    df = df.append(base_df, ignore_index=True, sort=False)


  # на этом этапе у нас df для фрейма наполнился одинаковыми строками
  # где только варианты различаются
  # ключ, соответствующий фрейму, лежит в fr
  # до кучи положим его туда же, в каждую строку
  df['key'] = [fr for i in range(0,len(frame['variants']))]
  # получившийся дф для фрейма приклеиваем к итоговому
  bl_df = bl_df.append(df, ignore_index=True, sort=False)
  # идём в следующий фрейм

# смотрим, сколько строчек вышло в дф и не потеряли ли мы что-то
len(bl_df)

6247

In [0]:
# дропнем лишние колонки, но две колонки комментариев хорошо бы было совместить конечно и оставить
bl_df.drop(['comment', 'comments', 'frames.value', 'title', 'variants'], axis=1, inplace=True)

In [5]:
# посмотрим, сколько всего повторяющихся вариантов предикатов
# их 196

bl_df.drop_duplicates(subset=['variant'], keep=False, inplace=True, ignore_index=True)
bl_df.variant.value_counts().head(196)

испытать благодарность      1
проявить сочувствие         1
нанести серьезный удар      1
стать дороже                1
раскручивание               1
                           ..
доверие                     1
атака войск                 1
внести предложение          1
выступать с предложением    1
покаяние                    1
Name: variant, Length: 196, dtype: int64

In [0]:
# ну, терь самое главное (типа)

# идем по строкам
for row in bl_df.itertuples():
  # достаём индес строки
  row_id = row[0]
  # и достаём из неё лист полярностей
  polarities_list = row[5]

  # пытаемся
  try:
    # для отдельной полярности
    for polarity in polarities_list:

      # обращаемся к элементам полярности (которая сама список) по индексу
      # элемент с индексом 0 - это тот, кто относится
      # элемент  индексом 1 - тот, к кому относятся
      # создаём название колонки формата 'КтоОтносится_ККому'
      col = '{}_{}'.format(polarity[0], polarity[1])

      #  обращаемся к ячейке по адресу [индекс строки, название новой колонки]
      # заполняем эту ячейку меткой (метка - это элемент полярности с индексом 2)
      # ну, метка бывает pos или neg
      # степень уверенности не берём с собой
      bl_df.loc[row_id, col] = polarity[2]


      # чтоб сохранить меру уверенности, можно её приклеить к метке,
      # если заменить строку выше на строку ниже
      # но в данный момент неясно, зачем она нужна, так что не будем
      # bl_df.loc[row_id, col] = '{} {}'.format(polarity[2], polarity[3])
  
  # если попытка не удалась, значит в полярности NaN
  except:
    pass
  # идём в следующую строку

In [0]:
bl_df['len'] = bl_df['variant'].apply(lambda x: len(x.split()))
bl_df['tag'] = bl_df['variant'].apply(lambda x: morph.parse(x)[0].tag.POS)

In [0]:
# то же самое для эффекта
# добавляем колонки 'effect_a1', 'effect_a0', 'effect_a2'

for row in bl_df.itertuples():
  row_id = row[0]
  effect_list = row[6]
  try:
    for effect in effect_list:
      col = 'effect_{}'.format(effect[0])
      bl_df.loc[row_id, col] = effect[1]
  except:
    pass

In [0]:
# то же самое для  state
# добавляем колонки 'state_a1', 'state_a0', 'state_a2'

for row in bl_df.itertuples():
  row_id = row[0]
  # семь - это номер колонки 'frames.state'
  state_list = row[7]
  try:
    for state in state_list:
      col = 'state_{}'.format(state[0])
      bl_df.loc[row_id, col] = state[1]
  except:
    pass

In [0]:
bl_df.state_a2.value_counts()

# a0
# pos    1785
# neg    1062

neg    225
pos     39
Name: state_a2, dtype: int64

In [0]:
bl_df.effect_a2.value_counts()
# a0
# +    935
# -    556

-    1160
+     775
Name: effect_a2, dtype: int64

In [0]:
bl_df.drop(['frames.polarity', 'frames.effect', 'frames.state'], axis=1, inplace=True)

In [10]:
bl_df.head()

,roles.a0,roles.a1,roles.a2,roles.a3,variant,key,a0_a1,a1_a2,a2_a1,author_a1,author_a2,a1_a0,a0_a2,author_a0,a2_a0,a0_a3,a1_a3,author_a3,a2_a2,a2_a3,effect_a1,effect_a0,effect_a2,effect_a3,state_a0,state_a1,state_a2
0,"тот, кто надеется","то, на что надеются",NaN,NaN,не терять надежду,0_0,pos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+,NaN,NaN,NaN,pos,NaN,NaN
1,"тот, кто надеется","то, на что надеются",NaN,NaN,ждать,0_0,pos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+,NaN,NaN,NaN,pos,NaN,NaN
2,"тот, кто надеется","то, на что надеются",NaN,NaN,чаять,0_0,pos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+,NaN,NaN,NaN,pos,NaN,NaN
3,"тот, кто надеется","то, на что надеются",NaN,NaN,тешить себя надеждой,0_0,pos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+,NaN,NaN,NaN,pos,NaN,NaN
4,"тот, кто надеется","то, на что надеются",NaN,NaN,питать надежду,0_0,pos,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,+,NaN,NaN,NaN,pos,NaN,NaN


# Странные effect и state

In [0]:
bl_df[
      (bl_df.effect_a1 == '+')
      &
      (bl_df.state_a1 == 'neg')
][['state_a1', 'effect_a1', 'variant', 'roles.a0', 'roles.a1', 'roles.a2', 'key']].head(1)

,state_a1,effect_a1,variant,roles.a0,roles.a1,roles.a2,key
5973,neg,+,выражать соболезнования,"тот, кто сочувствует","тот, кому сочувствует a0","то, в чем и из-за чего сочувствует a0",1_104


In [0]:
bl_data['1_104']

{'comment': '',
 'frames': {'effect': [['a1', '+', 1.0], ['a2', '-', 1.0]],
  'polarity': [['a0', 'a1', 'pos', 1.0],
   ['a0', 'a2', 'neg', 1.0],
   ['a1', 'a0', 'pos', 1.0],
   ['a1', 'a2', 'neg', 1.0],
   ['author', 'a0', 'pos', 1.0],
   ['author', 'a1', 'pos', 1.0],
   ['author', 'a2', 'neg', 1.0]],
  'state': [['a1', 'neg', 0.7]]},
 'roles': {'a0': 'тот, кто сочувствует',
  'a1': 'тот, кому сочувствует a0',
  'a2': 'то, в чем и из-за чего сочувствует a0'},
 'title': ['сочувствовать'],
 'variants': ['выражать соболезнования',
  'выражать сочувствие',
  'выразить соболезнования',
  'выразить сочувствие',
  'испытать сопереживание',
  'испытать сочувствие',
  'испытывать сопереживание',
  'испытывать сочувствие',
  'отнестись с сочувствием',
  'отнестись с участием',
  'относиться с сочувствием',
  'относиться с участием',
  'посочувствовать',
  'проявить сочувствие',
  'проявить участие',
  'проявлять сочувствие',
  'проявлять участие',
  'соболезнование',
  'соболезновать',
  'сопер

In [0]:
bl_df[
      (bl_df.effect_a1 == '-')
      &
      (bl_df.state_a1 == 'pos')
][['state_a1', 'effect_a1', 'variant', 'roles.a0', 'roles.a1', 'roles.a2', 'key']].head(1)

,state_a1,effect_a1,variant,roles.a0,roles.a1,roles.a2,key
5128,pos,-,заставать врасплох,"тот, кто застал и подловил","тот, кого застали и подловили","то, где или за чем застали и подловили",1_68


In [0]:
bl_data['1_68']

{'comment': '',
 'frames': {'effect': [['a1', '-', 1.0], ['a2', '-', 1.0]],
  'polarity': [['a0', 'a1', 'neg', 1.0],
   ['a1', 'a0', 'neg', 1.0],
   ['a0', 'a2', 'neg', 1.0],
   ['a1', 'a2', 'pos', 0.7],
   ['author', 'a0', 'pos', 0.7],
   ['author', 'a1', 'neg', 0.7],
   ['author', 'a2', 'neg', 0.7]],
  'state': [['a0', 'pos', 1.0], ['a1', 'pos', 1.0]]},
 'roles': {'a0': 'тот, кто застал и подловил',
  'a1': 'тот, кого застали и подловили',
  'a2': 'то, где или за чем застали и подловили'},
 'title': ['застать', 'подловить'],
 'variants': ['заставать врасплох',
  'заставать на месте преступления',
  'застать',
  'застать врасплох',
  'застать на месте преступления',
  'застигать',
  'застигать врасплох',
  'застигнуть врасплох',
  'застичь врасплох',
  'застукать',
  'ловить на месте преступления',
  'подлавливание',
  'подлавливать',
  'подловить',
  'поймать на месте преступления',
  'прихватить',
  'прихватить за жабры',
  'прихватывать',
  'прихватывать за жабры',
  'прищучивать',

#Достаём списком

In [0]:
bl_df.columns

Index(['roles.a0', 'roles.a1', 'roles.a2', 'roles.a3', 'frames.polarity',
       'frames.effect', 'frames.state', 'variant', 'key', 'n_polarities',
       'to_whom', 'score_mean', 'a0_a1', 'a1_a2', 'a2_a1', 'author_a1',
       'author_a2', 'a1_a0', 'a0_a2', 'a0_a3', 'a1_a3', 'author_a0', 'a2_a0',
       'author_a3', 'a2_a2', 'a2_a3', 'effect_a1', 'effect_a0', 'effect_a2',
       'effect_a3', 'state_a0', 'state_a1', 'state_a2'],
      dtype='object')

In [0]:
# тут по условию достаётся список глаголов
# раскаментим одно из двух условий ниже

# condition = 'pos'
condition = 'neg'

# по условиям достаём варианты
variants = bl_df[
      
      # первое условие
      (bl_df.a2_a1 == condition)
      
      # логическое И
      &
      
      # второе условие
      (bl_df.a1_a2 == condition)

      # можно добавить ещё миллион каких-нить условий вот в таком формате
      # &
      # (bl_df.a1_a2 == condition)

]['variant'].values

# нам интересны только те предикаты, которым пайморфи даёт pos-тег инфинитива
# или финитного глагола (на всякий)
# ну и чтобы предикат был одним словом, никаких пробелов
candidates = [variant for variant in variants 
              if 
              (' ' not in variant) 
              and 
              (('INFN' or 'VERB') in morph.parse(variant)[0].tag)
              ]
# имя файла будет с интересующей нас меткой в начале
fname = "{}_candidates_list.json".format(condition)

# сбрасываем лист кандидатов туда
with open(fname, "w") as write_file:
    json.dump(candidates, write_file, 
              # это чтоб нормально кириллица записалась
              ensure_ascii=False
              )

In [0]:
# делаем допущение, что так у нас получится вытащить глаголы с аргументом в дативе
# и второе допущение - что такие аргументы лежат в а1
# на самом деле мы поэкспериментировали с а0 и а2 и a3 тоже
# эксперименты показали, что лучше их в a1 искать

# заводим новую колонку 'to_whom'
# в ней будем хранить тру и фолс
# тру - мб датив есть
# фолс - мб его нету

# смотрим на описание роли a1
# если в нём есть ', кому' ИЛИ ', чему', лямбда-функция возвращает True
# и кладёт его в новую колонку
# ну и кладёт фолс если нету ни одной из этих строк в описании
# такого аргумента может и не быть, поэтому на всякий конвертируем его в строку
# эксепшены я не умею делать! научите...
bl_df['to_whom'] = bl_df['roles.a1'].apply(lambda x: (', кому' or ', чему') in str(x))

# в список кандидатов на датив кладём только нужные
# отсеиваем нужные по условиям
dative_candidates = [variant for variant
                    # во-первых, чтобы тру лежало в 'to_whom'
                     in bl_df[bl_df['to_whom'] == True]['variant'].values 
                    # во-вторых, чтоб это был глагол
                     if (' ' not in variant) and (('INFN' or 'VERB') in morph.parse(variant)[0].tag)]

# сбрасываем получившийся список в файл
with open("dative_candidates_list.json", "w") as write_file:
    json.dump(dative_candidates, write_file, ensure_ascii=False)

#Достаём словарём

In [0]:
# both positive and both negative

# достаём словарь вида {'глагол':'список полярностей'}
# если попадаются дубликаты, записывается полярность последнего

condition = 'pos'
# condition = 'neg'

conditioned_df = bl_df[
      (bl_df.a0_a1 == condition)
      &
      (bl_df.a1_a0 == condition)
      # можно добавить ещё миллион каких-нить условий вот в таком формате
      # &
      # (bl_df.a1_a0 == condition)
]

variants = conditioned_df['variant'].values
pols = conditioned_df['frames.polarity'].values

candidates = {k:v for (k,v) in dict(zip(variants, pols)).items() 
              if (' ' not in k)
              and (('INFN' or 'VERB') in morph.parse(k)[0].tag)
              }

fname = "{}_candidates_dict.json".format(condition)

with open(fname, "w") as write_file:
    json.dump(candidates, write_file, ensure_ascii=False)

In [0]:
# opposite polarities
# arguments

# достаём словарь вида {'глагол':'список полярностей'}
# если попадаются дубликаты, записывается полярность последнего

def conditioner(arg1, arg2, condition='opp', output_format='l', author=False):
  '''
  output_format: str, 'd' dictionary, 'l' list

  If author==True arg2 is destination of both author's and arg1's attitudes.
  '''
  if author==True:
    conditioned_df = bl_df[(bl_df['author_{}'.format(arg2)] != bl_df['{}_{}'.format(arg1, arg2)])
                          & (bl_df['author_{}'.format(arg2)].notnull())
                          & (bl_df['{}_{}'.format(arg1, arg2)].notnull())]
  else:
    conditioned_df = bl_df[(bl_df['{}_{}'.format(arg1, arg2)] != bl_df['{}_{}'.format(arg2, arg1)]) 
                          & (bl_df['{}_{}'.format(arg1, arg2)].notnull())
                          & (bl_df['{}_{}'.format(arg2, arg1)].notnull())]

  variants = conditioned_df['variant'].values

  if output_format == 'l':
    candidates = [variant for variant in variants 
                  if (' ' not in variant) 
                  and (('INFN' or 'VERB') in morph.parse(variant)[0].tag)]
  elif output_format == 'd':
    pols = conditioned_df['frames.polarity'].values
    candidates = {k:v for (k,v) in dict(zip(variants, pols)).items() 
                  if (' ' not in k)
                  and (('INFN' or 'VERB') in morph.parse(k)[0].tag)}
  if author==True:
    fname = "{}_author_{}_{}_candidates_dict.json".format(condition, arg1, arg2)
  else:
    fname = "{}_{}_{}_candidates_{}.json".format(condition, arg1, arg2, output_format)
  with open(fname, "w") as write_file:
      json.dump(candidates, write_file, ensure_ascii=False)
  return 0

# conditioner('a0', 'a1', 'opp', 'l')
conditioner('a0', 'a1', 'opp', 'l', author=True)

0

In [0]:
# opposite polarities
# случаи, когда различается отношение автора к аргументу и отношение другого аргумента к нему

# достаём словарь вида {'глагол':'список полярностей'}
# если попадаются дубликаты, записывается полярность последнего

condition = 'opp'

arg1 = 'a1'
arg2 = 'a0' 

conditioned_df = bl_df[
      (bl_df['author_{}'.format(arg2)] != bl_df['{}_{}'.format(arg1, arg2)])
      &
      (bl_df['author_{}'.format(arg2)].notnull())
      &
      (bl_df['{}_{}'.format(arg1, arg2)].notnull())
]

variants = conditioned_df['variant'].values
pols = conditioned_df['frames.polarity'].values

candidates = {k:v for (k,v) in dict(zip(variants, pols)).items() 
              if (' ' not in k)
              and (('INFN' or 'VERB') in morph.parse(k)[0].tag)
              }

fname = "{}_author_{}_{}_candidates_dict.json".format(condition, arg1, arg2)

with open(fname, "w") as write_file:
    json.dump(candidates, write_file, ensure_ascii=False)

In [0]:
# for (k,v) in candidates.items():
#   print (k,v)

In [0]:
# dictionary of dative candidates

bl_df['to_whom'] = bl_df['roles.a1'].apply(lambda x: (', кому' or ', чему') in str(x))

conditioned_df = bl_df[bl_df['to_whom'] == True]

variants = conditioned_df['variant'].values
pols = conditioned_df['frames.polarity'].values

candidates = {k:v for (k,v) in dict(zip(variants, pols)).items() 
              if (' ' not in k)
              and (('INFN' or 'VERB') in morph.parse(k)[0].tag)
              }

with open("dative_candidates_dict.json", "w") as write_file:
    json.dump(candidates, write_file, ensure_ascii=False)

# Пикл

In [0]:
# # в пикл записываем подопытный датафрейм
# bl_df.to_pickle('bl_df.pkl')

In [0]:
bl_df = pd.read_pickle("/content/bl_df.pkl")

#Будем группировать и посмотрим, что выйдет

In [0]:
bl_df.to_csv('collection.csv', index=False, encoding='utf-8', )

In [0]:
with open ('collection.csv', 'r', encoding='utf-8') as f:
    with open ('contrasting.csv', 'w', encoding='utf-8') as fw:
        for i, line in enumerate(f.readlines()):
            if i == 0:
                fw.write(line)
            if (('pos' or '+') in line) & (('neg' or '-') in line):
                fw.write(line)

In [100]:
len(bl_df)

5840

In [0]:
cont_df = pd.read_csv('contrasting.csv', encoding='utf-8', )

In [102]:
cont_df.columns

Index(['roles.a0', 'roles.a1', 'roles.a2', 'roles.a3', 'variant', 'key',
       'a0_a1', 'a1_a2', 'a2_a1', 'author_a1', 'author_a2', 'a1_a0', 'a0_a2',
       'author_a0', 'a2_a0', 'a0_a3', 'a1_a3', 'author_a3', 'a2_a2', 'a2_a3',
       'effect_a1', 'effect_a0', 'effect_a2', 'effect_a3', 'state_a0',
       'state_a1', 'state_a2', 'len', 'tag'],
      dtype='object')

In [103]:
no_author_df = cont_df[
          cont_df['author_a0'].isnull() &
          cont_df['author_a1'].isnull() &
          cont_df['author_a2'].isnull() &
          cont_df['author_a3'].isnull()
].drop(['author_a{}'.format(x) for x in range(0, 4)], axis=1)
print (len(no_author_df))

groups = [no_author_df['roles.a3'].notna(), 
          no_author_df['roles.a2'].notna(), 
          no_author_df['roles.a1'].notna(),
          no_author_df['roles.a0'].notna(),
        #   no_author_df['a2_a1'], 
        #   no_author_df['a1_a2'],
        #   no_author_df['a2_a0'], no_author_df['a0_a2'],


        #   no_author_df['roles.a1'].notna(), 
        #   no_author_df['roles.a0'].notna(), 

        #   no_author_df['a0_a1'], no_author_df['a1_a0'], 

          ]
no_author_df.groupby(by=groups).agg('nunique')

            #  ('' not in no_author_df['variant'])

            #   and (('INFN' or 'VERB') in morph.parse(k)[0].tag)

893


roles.a0  roles.a1  ...  len  tag
roles.a3 roles.a2 roles.a1 roles.a0                      ...          
False    False    True     True             7         7  ...    5    5
         True     True     True            31        31  ...    5    8
True     True     True     True             1         1  ...    1    2

[3 rows x 25 columns]

In [109]:
groups = [
          no_author_df['roles.a3'].notna(), 
          no_author_df['roles.a2'].notna(), 
          no_author_df['roles.a1'].notna(),
          no_author_df['roles.a0'].notna(),
        #   no_author_df['a2_a1'], 
        #   no_author_df['a1_a2'],
        #   no_author_df['a2_a0'], no_author_df['a0_a2'],
        #   no_author_df['roles.a1'].notna(), 
        #   no_author_df['roles.a0'].notna(), 
          no_author_df['a0_a1'], no_author_df['a1_a0'], 
          ]

no_author_df[
             no_author_df['len'] == 1 &
             ((no_author_df['tag']=='INFN')
             |(no_author_df['tag']=='VERB'))
             ].groupby(by=groups).agg('nunique')

roles.a0  roles.a1  ...  len  tag
roles.a3 roles.a2 roles.a1 roles.a0 a0_a1 a1_a0                      ...          
False    False    True     True     neg   neg           4         4  ...    1    1
         True     True     True     neg   neg          15        15  ...    1    1
                                    pos   pos           3         3  ...    1    1

[3 rows x 25 columns]

# Delete afterwards

In [0]:
print ('arg', 'pos', 'neg', '\n', sep='\t\t')
for arg in 'a0 a1 a2 a3'.split():
  pos = len(bl_df[bl_df['effect_{}'.format(arg)]=='+'])
  neg = len(bl_df[bl_df['effect_{}'.format(arg)]=='-'])
  print (arg, pos, neg, sep='\t\t')

arg		pos		neg		

a0		935		556
a1		2013		3288
a2		775		1160
a3		11		5
